In [13]:
!pip install bnlp-toolkit

In [14]:
from bnlp import NLTKTokenizer
tokenizer = NLTKTokenizer()

from tqdm import tqdm
from glob import glob
import pandas as pd
import os
import librosa

In [15]:
standard_bangla = pd.read_csv('/kaggle/input/bengaliai-train-csv/train.csv')

standard_bangla[['sentence']]

,sentence
0,ও বলেছে আপনার ঠিকানা!
1,কোন মহান রাষ্ট্রের নাগরিক হতে চাও?
2,"আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।"
3,নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন...
4,"হুমম, ওহ হেই, দেখো।"
...,...
963631,আপনার সাথে কথা বলতে চাই।
963632,সুতরাং পরের দিন আর-একটা ছবি না লইয়া চিত্রকর ছা...
963633,"সামাজিক কর্মকাণ্ডসমিতিতে গিয়ে দেখা যায়, শিল্পী..."
963634,গুগল ম্যাপসের সাহায্যে খুঁজে পাওয়া যাবে কোন জা...


In [16]:
token_list = []

for sen in standard_bangla['sentence']:
    tokens = tokenizer.word_tokenize(sen)
    token_list.append(tokens)


standard_bangla['Tokens'] = token_list


vocab_bangla = {}

for i in tqdm(standard_bangla.Tokens):
    for j in i:
        if j ==  "?" or j ==  "!" or j ==  "<" or j ==  ">" or j ==  "'" or j ==  "।" or j ==  "." or j ==  ",":
            pass
        else:
            try:
                vocab_bangla[j]+=1
            except:
                vocab_bangla[j]=1

standard_words = list(vocab_bangla.keys())

len(standard_words)

100%|██████████| 963636/963636 [00:05<00:00, 178071.72it/s]


181509

# Get Duration

In [17]:
def hms_format(seconds:float, explicit_format=False) -> str:
    """Returns seconds in hours minutes seconds format.

    Keyword argument:
        explicit_format: convert format from hh:mm:ss to hh hours mm minutes ss seconds
    """
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)

    if explicit_format:
        return "{} hours {:03} minutes {:03} seconds".format(int(hours), int(minutes), round(seconds))
    else:
        return "{}:{:02}:{:02}".format(int(hours), int(minutes), round(seconds))


def duration(base_dir,dataset,total_number_of_words_in_the_dataset):

    dataset_size = len(dataset)
    aud_list = list(dataset['file_name'])
    
    total_sec = 0
    aud_files_from_path = []
    
    for wav in tqdm(glob(os.path.join(base_dir, "*.*"))):
        aud_path = wav.split("/")
        aud_file = aud_path[-1]
        
        if aud_file in aud_list:
            aud_files_from_path.append(aud_file)
            d = librosa.get_duration(filename=wav)
            total_sec+=d
    
    
    hms = hms_format(seconds= total_sec, explicit_format=False)
    
    total_hours = round(total_sec/3600, 3)
    mint = total_sec//60

    avg_duration = round(total_sec/dataset_size, 3)
    wpm = round(total_number_of_words_in_the_dataset/mint, 3)
    wps = round(total_number_of_words_in_the_dataset/dataset_size, 3)

    return (total_sec,
            total_hours,
            avg_duration,
            wpm,
            hms,
            wps)

# Unique Words Functions

In [18]:
def unique_words(path, dis):
    
    if type(path) == str:
        dataset = pd.read_excel(path)
    
    else:
        dataset = path    

    dataset_size = len(dataset)
    
    district = dis

    token_list = []

    for sen in dataset['transcripts']:
        tokens = tokenizer.word_tokenize(str(sen))
        token_list.append(tokens)

    dataset['Tokens'] = token_list

    vocab = {}
    all_words = []

    for i in tqdm(dataset.Tokens):
        for j in i:
            if j ==  "?" or j ==  "!" or j ==  "<" or j ==  ">" or j ==  "'" or j ==  "।" or j ==  "." or j ==  ",":
                pass
            else:
                try:
                    vocab[j]+=1
                except:
                    vocab[j]=1

            all_words.append(j)


    tokenized_words = list(vocab.keys())
    total_number_words = sum(vocab.values())
    unique_words_frequency = len(vocab.keys())


    ood_dict = {}

    for i in tokenized_words:
        if i not in standard_words:
            if i in ood_dict:
                ood_dict[i] += 1
            else:
                ood_dict[i] = 1
            
    
    print(ood_dict)
    ood = list(ood_dict.keys())
    print(ood)
    dataset_of_ood_words = pd.DataFrame(ood,columns = [f'Unique words ({district})'])
    ood_words_unique = len(ood)
    #print(ood_words_unique)
    ood_words_total_freq = sum(ood_dict.values())
    #print(ood_words_total_freq)
    


    return (dataset,
            vocab,
            all_words,
            total_number_words,
            unique_words_frequency,
            dataset_of_ood_words,
            ood_words_total_freq,
            ood_words_unique,
            ood)

In [21]:
token_list = []

for sen in standard_bangla['sentence']:
    tokens = tokenizer.word_tokenize(sen)
    token_list.append(tokens)


standard_bangla['Tokens'] = token_list


vocab_bangla = {}

for i in tqdm(standard_bangla.Tokens):
    for j in i:
        if j ==  "?" or j ==  "!" or j ==  "<" or j ==  ">" or j ==  "'" or j ==  "।" or j ==  "." or j ==  ",":
            pass
        else:
            try:
                vocab_bangla[j]+=1
            except:
                vocab_bangla[j]=1

standard_words = list(vocab_bangla.keys())

len(standard_words)

100%|██████████| 963636/963636 [00:05<00:00, 175195.12it/s]


181509

In [20]:
# Generate standard words from Bengali dataset
print("Generating standard words...")
standard_bangla = pd.read_csv('/kaggle/input/bengaliai-train-csv/train.csv')
token_list = []
for sen in standard_bangla['sentence']:
    tokens = tokenizer.word_tokenize(sen)
    token_list.append(tokens)
    
vocab_bangla = {}
for i in tqdm(standard_bangla.Tokens):
    for j in i:
        if j not in ["?", "!", "<", ">", "'", "।", ".", ","]:
            try:
                vocab_bangla[j]+=1
            except:
                vocab_bangla[j]=1

standard_words = list(vocab_bangla.keys())
print(f"Total standard words: {len(standard_words)}")

Generating standard words...


AttributeError: 'DataFrame' object has no attribute 'Tokens'

In [23]:
# Generate standard words from Bengali dataset
print("Generating standard words...")
standard_bangla = pd.read_csv('/kaggle/input/bengaliai-train-csv/train.csv')
token_list = []
for sen in standard_bangla['sentence']:
    tokens = tokenizer.word_tokenize(sen)
    token_list.append(tokens)

# Add Tokens column to DataFrame - This was the missing step!
standard_bangla['Tokens'] = token_list
    
vocab_bangla = {}
for i in tqdm(standard_bangla.Tokens):
    for j in i:
        if j ==  "?" or j ==  "!" or j ==  "<" or j ==  ">" or j ==  "'" or j ==  "।" or j ==  "." or j ==  ",":
            pass
        else:
            try:
                vocab_bangla[j]+=1
            except:
                vocab_bangla[j]=1

standard_words = list(vocab_bangla.keys())
print(f"Total standard words: {len(standard_words)}")

def get_ood_words(dataset_path, dataset_type, district):
    """Extract OOD words from a dataset"""
    try:
        dataset = pd.read_excel(dataset_path)
        token_list = []
        
        for sen in dataset['transcripts']:
            tokens = tokenizer.word_tokenize(str(sen))
            token_list.append(tokens)
            
        vocab = {}
        for tokens in token_list:
            for token in tokens:
                if token not in ["?", "!", "<", ">", "'", "।", ".", ","]:
                    vocab[token] = vocab.get(token, 0) + 1
        
        ood_words = [word for word in vocab if word not in standard_words]
        return ood_words
        
    except Exception as e:
        print(f"Error processing {district} {dataset_type}: {str(e)}")
        return []

def create_ood_excel(ood_words_dict, output_path):
    """Create and save Excel file from OOD words dictionary"""
    max_words = max(len(words) for words in ood_words_dict.values())
    df_dict = {}
    
    for district in district_list:
        words = ood_words_dict[district]
        words_padded = words + [''] * (max_words - len(words))
        df_dict[district.lower()] = words_padded
    
    df = pd.DataFrame(df_dict)
    df.to_excel(output_path, index=False)
    print(f"Saved: {output_path}")
    return df

# District list
district_list = ["Rangpur", "Kishoreganj", "Narail", "Chittagong", "Narsingdi", 
                "Tangail", "Habiganj", "Barishal", "Sylhet", "Sandwip", "Comilla", "Noakhali"]

# Base directory path
base_dir = "/kaggle/input/interspeech-2025/district_wise"

# Dictionary to store OOD words for each district and dataset type
district_ood_words = {
    'test': {},
    'train': {},
    'valid': {},
    'combined': {}
}

# Process each district
for district in tqdm(district_list, desc="Processing districts"):
    # Process test data
    test_path = os.path.join(base_dir, district, f"{district}_test.xlsx")
    district_ood_words['test'][district] = get_ood_words(test_path, 'test', district)
    
    # Process train data
    train_path = os.path.join(base_dir, district, f"{district}_train.xlsx")
    district_ood_words['train'][district] = get_ood_words(train_path, 'train', district)
    
    # Process validation data
    valid_path = os.path.join(base_dir, district, f"{district}_valid.xlsx")
    district_ood_words['valid'][district] = get_ood_words(valid_path, 'valid', district)
    
    # Combine all OOD words for this district
    combined_words = set(district_ood_words['test'][district] + 
                        district_ood_words['train'][district] + 
                        district_ood_words['valid'][district])
    district_ood_words['combined'][district] = list(combined_words)

# Create Excel files
output_files = {
    'test': "/kaggle/working/district_ood_dictionary_test.xlsx",
    'train': "/kaggle/working/district_ood_dictionary_train.xlsx",
    'valid': "/kaggle/working/district_ood_dictionary_valid.xlsx",
    'combined': "/kaggle/working/district_ood_dictionary_combined.xlsx"
}

# Generate all Excel files
print("\nGenerating Excel files...")
for data_type, output_path in output_files.items():
    print(f"\nProcessing {data_type} dataset...")
    df = create_ood_excel(district_ood_words[data_type], output_path)
    print(f"First few rows of {data_type} dataset:")
    display(df.head())
    print("-" * 80)

Generating standard words...


100%|██████████| 963636/963636 [00:05<00:00, 169014.70it/s]


Total standard words: 181509


Processing districts: 100%|██████████| 12/12 [10:39<00:00, 53.32s/it]



Generating Excel files...

Processing test dataset...
Saved: /kaggle/working/district_ood_dictionary_test.xlsx
First few rows of test dataset:


...

--------------------------------------------------------------------------------

Processing valid dataset...
Saved: /kaggle/working/district_ood_dictionary_valid.xlsx
First few rows of valid dataset:


,rangpur,kishoreganj,narail,chittagong,narsingdi,tangail,habiganj,barishal,sylhet,sandwip,comilla,noakhali
0,কওয়া,কয়ডা,সগোলতা,গরি,ইতা,পুইড়া,মুটামুটি,আছিলাম,কিতাত,আঁই,দেশগুলা,আইবেন
1,জানোং,লেখছছ,সবতা,আইডিত,মুজিকালে,মার্কেটেও,আঁস,ধরছিলাম,কিতা,হইল্লে,ব্যাবসা-বানিজ্য,কইলেয়ে
2,সেইটায়,এইনো,ক্লহায়,অইলদি,তোকা,বঙ্গবাজারে,গিয়া,এট্টা,নিয়া,দিয়ালাইতো,নিয়া,তনিকা
3,চইলবার,তিনডা,কইছে,এইগিন,ফারছও,ইসলামীয়া,তুমরা,ব্যার,ফড়ো,কিরবো,দেক-শোন,ধোয়েছের
4,লাগচে,চাইরটা,আনিছিস,লইয়্যেরে,ফারি,মার্কেটগুলার,বাজারোতে,কচুরী,মহিলাত,হাইত্তান্ন,সাপ্তা,আঁর


--------------------------------------------------------------------------------

Processing combined dataset...
Saved: /kaggle/working/district_ood_dictionary_combined.xlsx
First few rows of combined dataset:


,rangpur,kishoreganj,narail,chittagong,narsingdi,tangail,habiganj,barishal,sylhet,sandwip,comilla,noakhali
0,খাছিনো,রাখছোত,কিনা-কাটা,আইছস,কমানি,ইংলট,দিতেনও,বেচছোস,রোহিঙ্গারে,ছড়া-টড়া,মরছিদেত্তে,হাচাইত্তর
1,ইদোত,খানয়ও,খুড়ো,কিনিয়ারে,করতারমু,দ্যাড়-দুই,ফাকেদা,কুডার,আটকাইতা,হোইলে,দানওয়কি,বিছনার
2,খাওয়াবে,আইছস,ফেলাইছো,লাইফত,আইছস,সমালোচনাডা,অফিসো,তোগোডা,বওয়াইছোইন,এমলাক,গুরুপ,কমতো
3,পঞ্চাস,অফিসো,ইমনরে,মুড়ির,গোস্তডা,ডেডিকেশন,গুন-গাষি,হ্যারও,ওয়ানো,কুরবাইন্না,গেছিগা,মৃৎশিল্ফের
4,আড়াইশো,ইয়তে,চালাতি,ফাকে,ডায়বিটিস,পীরগাছায়,ওয়ানো,এয়াগো,অইরাম,বেরাই-গোরাই,গডনা,গোই


--------------------------------------------------------------------------------


In [7]:
def get_phone_statistics(xlsx_path):
    # Read the Excel file
    df = pd.read_excel(xlsx_path)
    
    # 1. Calculate Characteristic Phones Pair Count
    def count_phone_pairs(text):
        # Convert Bengali text to a list of characters
        text = str(text)
        text = unicodedata.normalize('NFC', text)  # Normalize Unicode for Bengali
        chars = list(text)
        # Create pairs of consecutive characters
        pairs = [''.join(pair) for pair in zip(chars[:-1], chars[1:])]
        return len(set(pairs))
    
    phone_pair_count = df['transcripts'].dropna().apply(count_phone_pairs).mean()
    
    # 2. Calculate Average Phone Length Percentage
    def calc_phone_length(text):
        text = str(text)
        text = unicodedata.normalize('NFC', text)  # Normalize Unicode for Bengali
        phone_length = len(text.replace(' ', ''))  # Remove spaces
        text_length = len(text.strip())  # Trim and count total length
        return (phone_length / text_length) * 100 if text_length > 0 else 0
    
    avg_phone_length = df['transcripts'].dropna().apply(calc_phone_length).mean()
    
    # 3. Calculate Annotation Completeness
    annot_completeness = (df['transcripts'].notna() & (df['transcripts'].str.strip().str.len() > 0)).mean() * 100
    
    return phone_pair_count, avg_phone_length, annot_completeness

In [20]:
import unicodedata

# Experimenting

In [12]:
dis = 'Comilla'
path = '/kaggle/input/interspeech-2025/district_wise/Comilla/Comilla_test.xlsx'
base_dir = '/kaggle/input/interspeech-2025/district_wise/Comilla/test'

output_1 = unique_words(path, dis)

dataset = output_1[0]
dataset_size = len(dataset)
vocab_dict= output_1[1]
all_words_list = output_1[2]
total_number_of_words_in_the_dataset = output_1[3]
total_number_of_unique_words_in_the_dataset = output_1[4]
ood_words_dataset = output_1[5]
total_number_of_ood_words = output_1[6]
total_number_of_unique_ood_words = output_1[7]

oov_perc_tot = total_number_of_ood_words/total_number_of_words_in_the_dataset
oov_perc_unq = total_number_of_unique_ood_words/total_number_of_unique_words_in_the_dataset



output_2 = duration(base_dir,dataset,total_number_of_words_in_the_dataset)


print(f"###############################################{dis}###############################################")
print()
print(f'Total Samples {dataset_size}')
print(f"Total => {output_2[4]} ")
print(f"Average record size => {output_2[2]} seconds")
print(f"Words per minute => {output_2[3]} ")
print(f'Words per sample: {output_2[5]} words')
print(f'Total number of unique words present in {dis} dataset: {total_number_of_unique_words_in_the_dataset}')
print(f'OOD% in {dis} dataset (Unique): {round(oov_perc_unq*100, 3)} %')
print(f'Total number of words present in {dis} dataset: {total_number_of_words_in_the_dataset}')
print(f'Total number of OOD words present in {dis} dataset: {total_number_of_ood_words}')
print(f'Total number of unique OOD words present in {dis} dataset: {total_number_of_unique_ood_words}')

print(f'OOD% in {dis} dataset (Total): {round(oov_perc_tot*100, 3)} %')
print()
print(f"Total in seconds: {output_2[0]} seconds")
print(f"Total in Hours: {output_2[1]} hours")




100%|██████████| 32/32 [00:00<00:00, 38391.80it/s]


{'স্বর্ণকাররা': 1, 'স্বর্নকার': 1, 'হইছেগা': 1, 'ফাচানব্বই': 1, 'ওইয়া': 1, 'মাইনসের': 1, 'রূপাই': 1, 'বালো': 1, 'পিনতে': 1, 'পাচানব্বই': 1, 'টেয়ারও': 1, 'অহনে': 1, 'মেয়েরার': 1, 'বাড়ই': 1, 'ষাইড': 1, 'টেয়া': 1, 'নব্বইর': 1, 'অয়': 1, 'ছাইগা': 1, 'গেছেগা': 1, 'বাড়তেছে': 1, 'এইটুকই': 1, 'ওইদিনও': 1, 'মায়েরাই': 1, 'ছেলে-মেয়েদেরকে': 1, 'রুহিও': 1, 'ষোলোই': 1, 'রিয়ার্সেল': 1, 'ওইগুলার': 1, 'দশ-পনেরো': 1, 'আইয়ো': 1, 'মদ্যে': 1, 'দুলনা': 1, 'দোলনাতে': 1, 'যাইয়া': 1, 'লইছে': 1, 'ফ্রেন্ডটা': 1, 'মেয়েটার': 1, 'নুসাইবা': 1, 'নুসাইবার': 1, 'খেলা-ধুলা': 1, 'রিয়ার্সেলে': 1, 'ভাবলো': 1, 'ব্লুটুথে': 1, 'দোলনাটা': 1, 'ফিরা': 1, 'বান্ধুবি': 1, 'মদ্দে': 1, 'আছে।মানে': 1, 'চোখগুলা': 1, 'উল্টায়': 1, 'ছেলে-মেয়েরা': 1, 'করতাছেনা': 1, 'ভাবতেছে': 1, 'ছেলেগুলা': 1, 'দেখ-দেখ': 1, 'ভাবতাছে': 1, 'উইঠা': 1, 'রোদ্র': 1, 'দৌড়ায়া': 1, 'পারতেছে': 1, 'ফুইলা': 1, 'চিল্লান': 1, 'যেইভাবে': 1, 'ফরে': 1, 'নামাইছে': 1, 'নিয়া': 1, 'মেয়েরে': 1, 'এডা': 1, 'যাইতাছে': 1, 'মিল্লা': 1, 'ডে-শিফটের': 1, 'মরনিং': 1, 

100%|██████████| 32/32 [00:00<00:00, 47.90it/s]

###############################################Comilla###############################################

Total Samples 32
Total => 0:08:51 
Average record size => 16.582 seconds
Words per minute => 178.375 
Words per sample: 44.594 words
Total number of unique words present in Comilla dataset: 589
OOD% in Comilla dataset (Unique): 21.902 %
Total number of words present in Comilla dataset: 1427
Total number of OOD words present in Comilla dataset: 129
Total number of unique OOD words present in Comilla dataset: 129
OOD% in Comilla dataset (Total): 9.04 %

Total in seconds: 530.612 seconds
Total in Hours: 0.147 hours


# Main datasets

In [ ]:
rangpur_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/rangpur.xlsx")
kishoreganj_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/kishoreganj.xlsx")
narail_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/narail.xlsx")
chittagong_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/chittagong.xlsx")
narsingdi_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/narsingdi.xlsx")
tangail_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/tangail.xlsx")
habiganj_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/habiganj.xlsx")
barishal_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/barishal.xlsx")
sylhet_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/sylhet.xlsx")
sandwip_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/sandwip.xlsx")



dists = ["rangpur", "kishoreganj", "narail", "chittagong", "narsingdi", "tangail", "habiganj", "barishal", "sylhet", "sandwip"]

dis_words = {}

for dis in dists:
    
    if dis == "all":
        thesis_df = pd.concat([rangpur_df,kishoreganj_df,narail_df,chittagong_df,narsingdi_df], axis = 0)
        path = thesis_df
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/all'
        
    else:

        path = f'/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/{dis}.xlsx'
        base_dir = f'/kaggle/input/only-dis/only_dis/only_dis/{dis}/'

    dis = dis.capitalize()

    output_1 = unique_words(path, dis)
    
    dataset = output_1[0]
    dataset_size = len(dataset)
    vocab_dict= output_1[1]
    all_words_list = output_1[2]
    total_number_of_words_in_the_dataset = output_1[3]
    total_number_of_unique_words_in_the_dataset = output_1[4]
    ood_words_dataset = output_1[5]
    total_number_of_ood_words = output_1[6]
    total_number_of_unique_ood_words = output_1[7]
    dis_words[dis] = output_1[8]
    

    oov_perc_unq = total_number_of_unique_ood_words/total_number_of_unique_words_in_the_dataset
    oov_perc_tot = total_number_of_ood_words/total_number_of_words_in_the_dataset
 
    output_2 = duration(base_dir,dataset,total_number_of_words_in_the_dataset)


    print(f"====================================================== {dis} ======================================================")
    print()
    print()
    print()
    print(f'Total Samples {dataset_size}')
    print(f'Total number of words present in {dis} dataset: {total_number_of_words_in_the_dataset}')
    print(f'Total number of unique words present in {dis} dataset: {total_number_of_unique_words_in_the_dataset}')
    print(f'Total number of OOD words present in {dis} dataset: {total_number_of_ood_words}')
    print(f'Total number of unique OOD words present in {dis} dataset: {total_number_of_unique_ood_words}')
    print(f'OOD% in {dis} dataset (Unique words): {round(oov_perc_unq*100, 3)} %')
    print(f'OOD% in {dis} dataset (Total words): {round(oov_perc_tot*100, 3)} %')
    print()
    print(f"Total Duration: {output_2[4]}")
    print(f"Total in seconds: {output_2[0]} seconds")
    print(f"Total in Hours: {output_2[1]} hours")
    print(f"Average record size: {output_2[2]} seconds")
    print(f"Words per minute: {output_2[3]} ")
    print(f'Words per sample: {output_2[5]} words')
    print()
    print()
    print()
    print(f"###################################################################################################################")
    print()
    print()
    print()
    

output_df = pd.DataFrame()

for dis in dists:
    output_df = pd.concat(
        [output_df,
         pd.DataFrame({dis : dis_words[dis]})],
        axis=1
    )
output_df.to_excel("district_ood.xlsx", index=False)

# Subsets

In [ ]:
import pandas as pd

rangpur_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/rangpur.xlsx")
kishoreganj_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/kishoreganj.xlsx")
narail_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/narail.xlsx")
chittagong_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/chittagong.xlsx")
narsingdi_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/narsingdi.xlsx")
tangail_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/tangail.xlsx")
habiganj_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/habiganj.xlsx")
barishal_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/barishal.xlsx")
sylhet_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/sylhet.xlsx")
sandwip_df = pd.read_excel("/kaggle/input/only-dis/dis_only_dataframes/dis_only_dataframes/sandwip.xlsx")


rangpur_train = rangpur_df[rangpur_df["Split"] == "Train"]
rangpur_test = rangpur_df[rangpur_df["Split"] == "Test"]
rangpur_valid = rangpur_df[rangpur_df["Split"] == "Valid"]

kishoreganj_train = kishoreganj_df[kishoreganj_df["Split"] == "Train"]
kishoreganj_test = kishoreganj_df[kishoreganj_df["Split"] == "Test"]
kishoreganj_valid = kishoreganj_df[kishoreganj_df["Split"] == "Valid"]

narail_train = narail_df[narail_df["Split"] == "Train"]
narail_test = narail_df[narail_df["Split"] == "Test"]
narail_valid = narail_df[narail_df["Split"] == "Valid"]

chittagong_train = chittagong_df[chittagong_df["Split"] == "Train"]
chittagong_test = chittagong_df[chittagong_df["Split"] == "Test"]
chittagong_valid = chittagong_df[chittagong_df["Split"] == "Valid"]

narsingdi_train = narsingdi_df[narsingdi_df["Split"] == "Train"]
narsingdi_test = narsingdi_df[narsingdi_df["Split"] == "Test"]
narsingdi_valid = narsingdi_df[narsingdi_df["Split"] == "Valid"]

tangail_train = tangail_df[tangail_df["Split"] == "Train"]
tangail_test = tangail_df[tangail_df["Split"] == "Test"]
tangail_valid = tangail_df[tangail_df["Split"] == "Valid"]

habiganj_train = habiganj_df[habiganj_df["Split"] == "Train"]
habiganj_test = habiganj_df[habiganj_df["Split"] == "Test"]
habiganj_valid = habiganj_df[habiganj_df["Split"] == "Valid"]

barishal_train = barishal_df[barishal_df["Split"] == "Train"]
barishal_test = barishal_df[barishal_df["Split"] == "Test"]
barishal_valid = barishal_df[barishal_df["Split"] == "Valid"]

sylhet_train = sylhet_df[sylhet_df["Split"] == "Train"]
sylhet_test = sylhet_df[sylhet_df["Split"] == "Test"]
sylhet_valid = sylhet_df[sylhet_df["Split"] == "Valid"]

sandwip_train = sandwip_df[sandwip_df["Split"] == "Train"]
sandwip_test = sandwip_df[sandwip_df["Split"] == "Test"]
sandwip_valid = sandwip_df[sandwip_df["Split"] == "Valid"]


########################################################################

all_df = pd.concat([rangpur_df,kishoreganj_df,narail_df,chittagong_df,narsingdi_df,
                    tangail_df,habiganj_df,barishal_df,sylhet_df,sandwip_df], axis = 0)

all_train = all_df[all_df["Split"] == "Train"]
all_test = all_df[all_df["Split"] == "Test"]
all_valid = all_df[all_df["Split"] == "Valid"]

In [ ]:
all_df_thesis = pd.concat([rangpur_df,kishoreganj_df,narail_df,chittagong_df,narsingdi_df], axis = 0)


all_train_thesis = all_df_thesis[all_df_thesis["Split"] == "Train"]
all_test_thesis = all_df_thesis[all_df_thesis["Split"] == "Test"]
all_valid_thesis = all_df_thesis[all_df_thesis["Split"] == "Valid"]

# Train

In [ ]:
trains = [rangpur_train, kishoreganj_train, narail_train, chittagong_train, narsingdi_train, all_train]
dists = ["rangpur", "kishoreganj", "narail", "chittagong", "narsingdi", "all"]


for dis in dists:
    
    if dis == 'rangpur':
        df = rangpur_train
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'kishoreganj':
        df = kishoreganj_train
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'narail':
        df = narail_train
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'chittagong':
        df = chittagong_train
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'narsingdi':
        df = narsingdi_train
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'all':
        df = all_train_thesis
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/splits/train/'
    

    dis = dis.capitalize()

    output_1 = unique_words(df, dis)
    
    dataset = output_1[0]
    dataset_size = len(dataset)
    vocab_dict= output_1[1]
    all_words_list = output_1[2]
    total_number_of_words_in_the_dataset = output_1[3]
    total_number_of_unique_words_in_the_dataset = output_1[4]
    ood_words_dataset = output_1[5]
    total_number_of_ood_words = output_1[6]
    total_number_of_unique_ood_words = output_1[7]

    oov_perc_unq = total_number_of_unique_ood_words/total_number_of_unique_words_in_the_dataset
    oov_perc_tot = total_number_of_ood_words/total_number_of_words_in_the_dataset
 
    output_2 = duration(base_dir,dataset,total_number_of_words_in_the_dataset)


    print(f"====================================================== {dis} ======================================================")
    print()
    print(f'Total Samples {dataset_size}')
    print(f'Total number of words present in {dis} dataset: {total_number_of_words_in_the_dataset}')
    print(f'Total number of unique words present in {dis} dataset: {total_number_of_unique_words_in_the_dataset}')
    print(f'Total number of OOD words present in {dis} dataset: {total_number_of_ood_words}')
    print(f'Total number of unique OOD words present in {dis} dataset: {total_number_of_unique_ood_words}')
    print(f'OOD% in {dis} dataset (Unique words): {round(oov_perc_unq*100, 3)} %')
    print(f'OOD% in {dis} dataset (Total words): {round(oov_perc_tot*100, 3)} %')
    print()
    print(f"Total Duration: {output_2[4]}")
    print(f"Total in seconds: {output_2[0]} seconds")
    print(f"Total in Hours: {output_2[1]} hours")
    print(f"Average record size: {output_2[2]} seconds")
    print(f"Words per minute: {output_2[3]} ")
    print(f'Words per sample: {output_2[5]} words')
    print()
    print()
    print()
    print(f"###################################################################################################################")
    print()
    print()


# Test

In [ ]:
trains = [rangpur_test, kishoreganj_test, narail_test, chittagong_test, narsingdi_test, all_test]
dists = ["rangpur", "kishoreganj", "narail", "chittagong", "narsingdi", "all"]

for dis in dists:
    
    if dis == 'rangpur':
        df = rangpur_test
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'kishoreganj':
        df = kishoreganj_test
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'narail':
        df = narail_test
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'chittagong':
        df = chittagong_test
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'narsingdi':
        df = narsingdi_test
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'all':
        df = all_test_thesis
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/splits/test/'
    

    dis = dis.capitalize()

    output_1 = unique_words(df, dis)
    
    dataset = output_1[0]
    dataset_size = len(dataset)
    vocab_dict= output_1[1]
    all_words_list = output_1[2]
    total_number_of_words_in_the_dataset = output_1[3]
    total_number_of_unique_words_in_the_dataset = output_1[4]
    ood_words_dataset = output_1[5]
    total_number_of_ood_words = output_1[6]
    total_number_of_unique_ood_words = output_1[7]

    oov_perc_unq = total_number_of_unique_ood_words/total_number_of_unique_words_in_the_dataset
    oov_perc_tot = total_number_of_ood_words/total_number_of_words_in_the_dataset
 
    output_2 = duration(base_dir,dataset,total_number_of_words_in_the_dataset)


    print(f"====================================================== {dis} ======================================================")
    print()
    print(f'Total Samples {dataset_size}')
    print(f'Total number of words present in {dis} dataset: {total_number_of_words_in_the_dataset}')
    print(f'Total number of unique words present in {dis} dataset: {total_number_of_unique_words_in_the_dataset}')
    print(f'Total number of OOD words present in {dis} dataset: {total_number_of_ood_words}')
    print(f'Total number of unique OOD words present in {dis} dataset: {total_number_of_unique_ood_words}')
    print(f'OOD% in {dis} dataset (Unique words): {round(oov_perc_unq*100, 3)} %')
    print(f'OOD% in {dis} dataset (Total words): {round(oov_perc_tot*100, 3)} %')
    print()
    print(f"Total Duration: {output_2[4]}")
    print(f"Total in seconds: {output_2[0]} seconds")
    print(f"Total in Hours: {output_2[1]} hours")
    print(f"Average record size: {output_2[2]} seconds")
    print(f"Words per minute: {output_2[3]} ")
    print(f'Words per sample: {output_2[5]} words')
    print()
    print()
    print()
    print(f"###################################################################################################################")
    print()
    print()


# Valid

In [ ]:
trains = [rangpur_valid, kishoreganj_valid, narail_valid, chittagong_valid, narsingdi_valid, all_valid]
dists = ["rangpur", "kishoreganj", "narail", "chittagong", "narsingdi", "all"]

for dis in dists:
    
    if dis == 'rangpur':
        df = rangpur_valid
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'kishoreganj':
        df = kishoreganj_valid
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'narail':
        df = narail_valid
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'chittagong':
        df = chittagong_valid
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'narsingdi':
        df = narsingdi_valid
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/{dis}/'
    elif dis == 'all':
        df = all_valid_thesis
        base_dir = f'/kaggle/input/thesis-copy/thesis copy/audios/splits/valid/'
    

    dis = dis.capitalize()

    output_1 = unique_words(df, dis)
    
    dataset = output_1[0]
    dataset_size = len(dataset)
    vocab_dict= output_1[1]
    all_words_list = output_1[2]
    total_number_of_words_in_the_dataset = output_1[3]
    total_number_of_unique_words_in_the_dataset = output_1[4]
    ood_words_dataset = output_1[5]
    total_number_of_ood_words = output_1[6]
    total_number_of_unique_ood_words = output_1[7]

    oov_perc_unq = total_number_of_unique_ood_words/total_number_of_unique_words_in_the_dataset
    oov_perc_tot = total_number_of_ood_words/total_number_of_words_in_the_dataset
 
    output_2 = duration(base_dir,dataset,total_number_of_words_in_the_dataset)


    print(f"====================================================== {dis} ======================================================")
    print()
    print(f'Total Samples {dataset_size}')
    print(f'Total number of words present in {dis} dataset: {total_number_of_words_in_the_dataset}')
    print(f'Total number of unique words present in {dis} dataset: {total_number_of_unique_words_in_the_dataset}')
    print(f'Total number of OOD words present in {dis} dataset: {total_number_of_ood_words}')
    print(f'Total number of unique OOD words present in {dis} dataset: {total_number_of_unique_ood_words}')
    print(f'OOD% in {dis} dataset (Unique words): {round(oov_perc_unq*100, 3)} %')
    print(f'OOD% in {dis} dataset (Total words): {round(oov_perc_tot*100, 3)} %')
    print()
    print(f"Total Duration: {output_2[4]}")
    print(f"Total in seconds: {output_2[0]} seconds")
    print(f"Total in Hours: {output_2[1]} hours")
    print(f"Average record size: {output_2[2]} seconds")
    print(f"Words per minute: {output_2[3]} ")
    print(f'Words per sample: {output_2[5]} words')
    print()
    print()
    print()
    print(f"###################################################################################################################")
    print()
    print()
